You need to make sure you have tika and get the docker image running with this command: 

IMAGE=tgowda/rtg-model:500toEng-v1

docker run --platform linux/amd64 --rm -i -p 6060:6060 $IMAGE

I added in  --platform Linux/amd64  to the command bc I’m on a mac and I needed that to run the image 

You’ll also need to pip install tika and emoji. I accidentally installed / imported requests too but didn’t end up using it 

Please make sure you have Java1.8 or higher intsalled on the environment you are using to run the notebook

In [1]:
pip install tika

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install emoji

Note: you may need to restart the kernel to use updated packages.


In [77]:
import pandas as pd
from tika import translate
from tika import language
import requests
import re
import emoji


In [78]:
df = pd.read_csv('/Users/taniadawood/Desktop/DSCI550-PixstoryMediaExtractionAndAnalysis/1_Language Identification/Master_Dataset_Raw_LangDetect_Complete.csv')


In [79]:
df['Narrative'] = df['Narrative'].astype(str)

In [ ]:
#taken from the emoji docs since it's not in the latest versions
def get_emoji_regexp():
    # Sort emoji by length to make sure multi-character emojis are
    # matched first
    emojis = sorted(emoji.EMOJI_DATA, key=len, reverse=True)
    pattern = u'(' + u'|'.join(re.escape(u) for u in emojis) + u')'
    return re.compile(pattern)

exp = get_emoji_regexp()

#this function removes any emojis, html tags, URLs, and special characters before translating
def clean_text(text):
    try:
        text = exp.sub(u'', text)
        text = re.sub(r'<.*?>', '', text)
        text = re.sub(r'http\S+', '', text)
        text = re.sub('[^a-zA-Z]+', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        return text.strip()
    except :
        print(text)
    finally:
        return text
    
    
clean_narr = clean_text(df['Narrative'].apply(clean_text))

df['Clean Narrative'] = clean_narr

‌ Counting of Votes of assembly elections will commence tomorrow 
‌
‌UP: Lucknow Police Commissioner DK Thakur said following allegations of EVM tampering, "We've adequate security measures in place at EVM strong room. The strong room will be opened around 7 am in the presence of election observers and candidates; Counting will begin at 8 am tomorrow. Police patrolling will be done in sensitive areas."
‌
‌Uttarkhand: The Congress agreed that during the counting, senior party leaders would be present to keep an eye on the counting of votes in all thirteen districts of the state, as per a report by ANI.
‌
‌Punjab: Laddu preparation on full swing ahead of the counting day of state assembly elections on March 10 in Ludhiana.
‌
‌Ahead of vote counting on March 10, Samajwadi Party president Akhilesh Yadav has put up questions on the security of the Electronic Voting Machines (EVMs). He accused the Election Commission officials of tampering with the EVMs and said that he no longer trusts the 

Il comunicato di Trenitalia: 

Sarà riattivata alle 14.30 di oggi la linea ad Alta Velocità Roma-Napoli, con la circolazione dei treni ad alta velocità in graduale ripresa già dal pomeriggio di oggi. Questo consentirà anche il miglioramento della circolazione dei treni pendolari sulle linee convenzionali.
 
Nella mattina di domani, mercoledì 8 giugno, potrà riprendere la normale programmazione dei treni sia fra Roma e Napoli sia nei nodi ferroviari delle due città.
 
La linea era stata interrotta nel pomeriggio di venerdì 3 giugno a seguito dello svio della locomotiva di coda di un treno AV nei pressi di Roma Prenestina.
 
Oltre 70 tecnici di Rete Ferroviaria Italiana (Gruppo FS Italiane) hanno lavorato per rinnovare circa un chilometro e trecento metri di binari, sostituire 400 metri di cavi tecnologici e ripristinare 150 metri di linea di alimentazione elettrica dei treni.
 
Le attività di ripristino, concluse in meno di 48 ore, sono potute avvenire in due step: il primo è iniziato d

Eun Jeong Ko Su JungLee 

Abstract 

Purpose 

This study aimed to assess indicators of type 2 diabetes mellitus (DM) management, including adequate DM control, and treatment rates, in cancer survivors according to the time of DM diagnosis and to compare them with the DM management indicators of a non-cancer control group. 

Methods 

We used the 2013–2019 data of the Korea National Health and Nutrition Examination Survey for this study. To compare their adequate DM control, and treatment rates, we identified 4,918 patients with type 2 DM aged ≥ 30 years and classified them into pre-existing diabetes, pre-existing cancer, and diabetes without cancer groups. Predictors of adequate glycemic control and diabetes treatment were analyzed using binary logistic regression. 

Results 

Diabetes without cancer group had higher fasting blood glucose and glycosylated hemoglobin A1c levels and lower adequate glycemic control than did the other two groups. The preexisting cancer group had low treat

In [ ]:
chars = 10 #specifies the number of characters to translate out of the narrative

translated_column = []
for text, language in zip(df['Clean Narrative'], df['Narrative TikaDetect']):
    if language != 'en':
        if len(text) > chars:
            translated_chunks = []
            try:
                #translate the narrative in chunks depending on the limit set and join them all together at the end
                for i in range(0, len(text), chars):
                    group = text[i:i+chars]
                    translation = translate.from_buffer(group, language, 'en', requestOptions={'timeout':500})
                    if ((translation == '') or (not translation)):
                        raise ValueError('Tika server returned 500')
                    translated_chunks.append(translation)
                translated_text = ' '.join(translated_chunks)
                translated_column.append(translated_text)
                
            except Exception as error:
                print(error)
                print('error text', text)
                #Appending "redo" for narratives that fail so they will be searchable to run again and update
                translated_column.append('redo')
        else:
            try:
                #translate the entire narrative if it's under the specified char size
                translated_text = translate.from_buffer(text, language, 'en', requestOptions={'timeout':500})
                if ((translated_text == '') or (not translated_text)):
                    raise ValueError('Tika server returned 500')
                translated_column.append(translated_text)
            except Exception as error:
                print(error)
                print('error text', text)
                #Appending "redo" for narratives that fail so they will be searchable to run again and update
                translated_column.append('redo')
    else:
        #just append the regular text if it's already english
        translated_column.append(text)

In [ ]:
df['Translated Narrative'] = translated_column

In [ ]:
df.to_csv('Master_Dataset_Raw_LangDetect_Translated.csv', index=False)